In [1]:
import fenics as fe
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Define mesh and function space
mesh = fe.UnitSquareMesh(128, 128)
V = fe.FunctionSpace(mesh, "Lagrange", 1)

# Define subdomains
class INNERDomain(fe.SubDomain):
    def inside(self, x, on_boundary):
        return ((x[0]-0.5)*(x[0]-0.5)+(x[1]-0.5)*(x[1]-0.5)) < 0.25*0.25 

class OUTDomain(fe.SubDomain):
    def inside(self, x, on_boundary):
        return ((x[0]-0.5)*(x[0]-0.5)+(x[1]-0.5)*(x[1]-0.5)) >= 0.25*0.25  

# Create MeshFunction to mark subdomains
subdomains = fe.MeshFunction("size_t", mesh, mesh.topology().dim())
INNER_domain = INNERDomain()
OUT_domain = OUTDomain()
INNER_domain.mark(subdomains, 0)
OUT_domain.mark(subdomains, 1)

# Parameters for the Gaussian function
A = 1.0        
x0, y0 = 0.5, 0.5  
sigma = 0.01 
B = 1.0
sigma2 = 10

# Define different diffusion coefficients for each subdomain
D = fe.Expression("((x[0]-0.5)*(x[0]-0.5)+(x[1]-0.5)*(x[1]-0.5)) < 0.25*0.25 ? A * exp(-((x[0] - x0)*(x[0] - x0) + (x[1] - y0)*(x[1] - y0)) / (2 * sigma)) \
                    : B * exp(-((x[0] - x0)*(x[0] - x0) + (x[1] - y0)*(x[1] - y0)) / (2 * sigma2))",
                  degree=2, A=A, x0=x0, y0=y0, sigma=sigma, B=B, sigma2=sigma2, domain=mesh)

# Define boundary condition
def boundary_boolean_function(x, on_boundary):
    return on_boundary

bc = fe.DirichletBC(V, fe.Constant(0.0), boundary_boolean_function)

# The initial condition
parameter = 10
x0, y0 = 0.5,0.5  # Center of the initial condition in the unit square
magnitude = 1
initial_condition = fe.Expression("magnitude*(1-exp(-1/((parameter*parameter*(x[0]-x0)*(x[0]-x0)+parameter*parameter*(x[1]-y0)*(x[1]-y0)))))", 
                                  degree=2, magnitude=magnitude, parameter=parameter, x0=x0, y0=y0, domain=mesh)

# Discretize the initial condition
u_old = fe.interpolate(initial_condition, V)

# Define variational problem
u_trial = fe.TrialFunction(V)
v_test = fe.TestFunction(V)
# Time-stepping length
time_step_length = 0.001
# Time-stepping loop
n_time_steps = 200

# Weak form with piecewise diffusion coefficient
weak_form_residuum = (
    (u_trial * v_test * fe.dx
    +
    time_step_length * (fe.dot(fe.grad(u_trial), fe.grad(D)) * v_test 
                        +
                        D * fe.dot(fe.grad(u_trial), fe.grad(v_test))) * fe.dx)
    -
    (
        u_old * v_test * fe.dx
        +
        time_step_length * fe.dot(fe.grad(D), fe.grad(u_trial)) * v_test * fe.dx
    ))

#We have a linear PDE that is separable into a lhs and rhs
weak_form_lhs = fe.lhs(weak_form_residuum)
weak_form_rhs = fe.rhs(weak_form_residuum)

# Solution function for each timestep
u_solution = fe.Function(V)

# Set up file for saving results
vtkfile = fe.File("Circle_domians/solution.pvd")

# Time-stepping loop
time_current = 0.0
for i in range(n_time_steps):
    time_current += time_step_length
    # Solve the PDE for the current time step
    fe.solve(weak_form_lhs == weak_form_rhs, u_solution, bc)
    # Update the previous solution
    u_old.assign(u_solution)
    vtkfile << (u_solution, time_current)
    # Save the current solution to the .pvd file

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational p